## Import Package

In [1]:
# required packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import numba as nb
import warnings
from datetime import datetime
import gc
from utils import *

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error


# display set up
%matplotlib inline
%precision 4
warnings.filterwarnings('ignore')
np.set_printoptions(suppress=True)
pd.set_option("display.precision", 3)
pd.options.display.max_rows = 999

## Load Data

In [4]:
%%time

prop_2016 = load_data('data/prop_2016_v2.csv')
prop_2017 = load_data('data/prop_2017_v2.csv')
train = load_data('data/train_v2.csv')

Mem. usage decreased to 999.27 Mb (75.6% reduction)
Mem. usage decreased to 999.27 Mb (75.6% reduction)
Mem. usage decreased to 58.44 Mb (75.3% reduction)
CPU times: user 4min 28s, sys: 3min 10s, total: 7min 38s
Wall time: 9min


## Modeling Prep

In [5]:
%%time
def encoding(train, prop_2016, prop_2017, categorical_features):
    """
    To limit the value to small numbers since values in categorical_feature is suggested to be small.
    Now the number denoting missing values will be 0 instead of -1.
    """
    from category_encoders import BinaryEncoder
    
    split_index = [len(train), len(prop_2016), len(prop_2017)]
    
    
    prop = pd.concat([train[categorical_features],
                      prop_2016[categorical_features], 
                      prop_2017[categorical_features]], ignore_index=True)
    
    encoder = BinaryEncoder(cols=categorical_features, drop_invariant = True)
    transformed = encoder.fit_transform(prop)
    train_be = transformed.iloc[: split_index[0]].reset_index(drop=True)
    prop_2016_be = transformed.iloc[split_index[0] : split_index[0]+split_index[1]].reset_index(drop=True)
    prop_2017_be = transformed.iloc[split_index[0]+split_index[1]:].reset_index(drop=True)
    

    train = pd.concat([train.drop(columns=categorical_features).reset_index(drop=True), train_be], axis = 1)
    prop_2016 = pd.concat([prop_2016.drop(columns=categorical_features).reset_index(drop=True), prop_2016_be], axis = 1)
    prop_2017 = pd.concat([prop_2017.drop(columns=categorical_features).reset_index(drop=True), prop_2017_be], axis = 1)

    
    del prop, transformed, train_be, prop_2016_be, prop_2017_be
    gc.collect()
    return(train, prop_2016, prop_2017)

# Do encoding for cateogircal features
categorical_features = ['airconditioningtypeid', 'architecturalstyletypeid', 'buildingclasstypeid', 'buildingqualitytypeid',
              'fips', 'heatingorsystemtypeid', 'propertycountylandusecode', 'propertylandusetypeid',
              'propertyzoningdesc', 'regionidcity', 'regionidcounty', 'regionidneighborhood', 'regionidzip',
              'storytypeid', 'typeconstructiontypeid', 'geo_cluster']

train, prop_2016, prop_2017 = encoding(train, prop_2016, prop_2017, categorical_features)

CPU times: user 1min 38s, sys: 1min 29s, total: 3min 7s
Wall time: 3min 46s


In [6]:
# Transform to Numpy matrices
X = train.drop(columns=['logerror', 'parcelid', 'transactiondate'])
y = train.logerror.values


# Perform shuffled train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

print("X_train shape: {}".format(X_train.shape))
print("y_train shape: {}".format(y_train.shape))
print("X_val shape: {}".format(X_test.shape))
print("y_val shape: {}".format(y_test.shape))

X_train shape: (134310, 214)
y_train shape: (134310,)
X_val shape: (33578, 214)
y_val shape: (33578,)


In [55]:
## parameter chosen
params = {'bootstrap': True,
          'ccp_alpha': 0.0000,
          'criterion': 'mse',
          'max_depth': 3,
          'max_features': 'auto',
          'max_leaf_nodes': 9,
          'max_samples': None,
          'min_impurity_decrease': 0.0051,
          'min_impurity_split': None,
          'min_samples_leaf': 6,
          'min_samples_split': 12,
          'min_weight_fraction_leaf': 0.0000,
          'n_estimators': 10,
          'n_jobs': None,
          'oob_score': False,
          'random_state': None,
          'verbose': 0,
          'warm_start': False}

{'bootstrap': True,
 'ccp_alpha': 0.0000,
 'criterion': 'mse',
 'max_depth': 3,
 'max_features': 'auto',
 'max_leaf_nodes': 9,
 'max_samples': None,
 'min_impurity_decrease': 0.0051,
 'min_impurity_split': None,
 'min_samples_leaf': 6,
 'min_samples_split': 12,
 'min_weight_fraction_leaf': 0.0000,
 'n_estimators': 10,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

## Training on all Data

In [58]:
# Train multiple models
iterations = 3
models = []
params['n_estimators'] = 100
params['verbose'] = 2
for i in range(iterations):
    print("Start training model {}".format(i))
    params['random_state'] = (i+1)*100
    model = RandomForestRegressor(**params)
    model.fit(X, y)
    models.append(model)
    
# Sanity check (make sure scores on a small portion of the dataset are reasonable)
for i, model in enumerate(models):
    print("model {}: {}".format(i, abs(model.predict(X_test) - y_test).mean() * 100))

Start training model 0


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s


building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100


...

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   59.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


model 0: 7.191401230321529
model 1: 7.191342800752031


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


model 2: 7.191235857471194


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


In [59]:
def train_predict_and_export(models, prop_2016, prop_2017, file_name):
    """
    Helper method to make predicition and export results to csv.
    """
    # Construct DataFrame for prediction results
    submission_2016 = pd.DataFrame()
    submission_2017 = pd.DataFrame()
    submission_2016['ParcelId'] = prop_2016.parcelid
    submission_2017['ParcelId'] = prop_2017.parcelid

    # Make Prediction across multiple models
    pred_2016, pred_2017 = [], []
    for i, model in enumerate(models):
        print("Start model {} (2016)".format(i))
        pred_2016.append(model.predict(prop_2016))
        print("Start model {} (2017)".format(i))
        pred_2017.append(model.predict(prop_2017))
    
    # Take average across all models
    mean_pred_2016 = np.mean(pred_2016, axis=0)
    mean_pred_2017 = np.mean(pred_2017, axis=0)
    
    # Formatting for submission
    submission_2016['201610'] = [float(format(x, '.4f')) for x in mean_pred_2016]
    submission_2016['201611'] = submission_2016['201610']
    submission_2016['201612'] = submission_2016['201610']

    submission_2017['201710'] = [float(format(x, '.4f')) for x in mean_pred_2017]
    submission_2017['201711'] = submission_2017['201710']
    submission_2017['201712'] = submission_2017['201710']
    
    submission = pd.merge(submission_2016,submission_2017, how='inner', on='ParcelId')
    submission.to_csv(file_name, index=False)
    print("Submission Successfully Created.")
    return(submission, pred_2016, pred_2017)

In [60]:
%%time
del train
gc.collect()

file_name = 'submission/result_rf.csv'
submission, pred_2016, pred_2017 = predict_and_export(models, prop_2016, prop_2017, file_name)

Start model 0 (2016)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.5s finished


Start model 0 (2017)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    3.0s finished


Start model 1 (2016)


...

Start model 2 (2016)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.2s finished


Start model 2 (2017)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.1s finished


Submission Successfully Created.
CPU times: user 58.1 s, sys: 54.5 s, total: 1min 52s
Wall time: 2min 23s
